In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dask
import dask.dataframe as dd

from sklearn.model_selection import GridSearchCV, KFold,KFold, cross_val_score

from sklearn.preprocessing import PolynomialFeatures, StandardScaler,RobustScaler 

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.linear_model import LogisticRegression,LinearRegression,Lasso

from sklearn.metrics import accuracy_score, f1_score,mean_squared_error,classification_report,confusion_matrix

from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dask
import dask.dataframe as dd

import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import GridSearchCV, KFold,KFold, cross_val_score

from sklearn.preprocessing import PolynomialFeatures, StandardScaler,RobustScaler 

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.linear_model import LogisticRegression,LinearRegression,Lasso

from sklearn.metrics import accuracy_score, f1_score,mean_squared_error,classification_report,confusion_matrix

from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import make_column_transformer

from scipy import stats
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline

In [2]:
df = pd.read_pickle("data_after_normalization-v1.pickle")

In [3]:
df = df.drop(['Charging_Time_in_hours', 'Precipitation','Temperature','pd','Bahnhof','Restaurant'
              ,'Biergarten','Community_Centre','Hospital','Kino','Parking','university','Charging_time_in_hours_log_New'], axis = 1)

In [4]:
df = df.rename(columns = {'Temperature_New':'Temperature', 'Precipitation_New':'Precipitation',
                          'pd_New':'pd','Bahnhof_New':'Bahnhof','Restaurant_New':'Restaurant','Biergarten_New':'Biergarten',
                          'Community_Centre_New':'Community_Centre', 'Hospital_New':'Hospital',
                         'Kino_New':'Kino', 'Parking_New':'Parking','university_New':'university', 'Charging_Time_in_hours_New':'Charging_Time_in_hours',
                          })

In [5]:
df['Year'] = pd.to_datetime(df['datetime_start'], format='%Y')

In [6]:
df['Month'] = pd.to_datetime(df['datetime_start'], format='%M')

In [7]:
df['Day'] = pd.to_datetime(df['datetime_start'], format='%D')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1796287 entries, 0 to 1940933
Data columns (total 31 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   park_id                     object        
 1   clc_code                    object        
 2   area_code                   object        
 3   evse_id                     object        
 4   datetime_start              datetime64[ns]
 5   datetime_stop               datetime64[ns]
 6   city                        object        
 7   latitude                    float64       
 8   longitude                   float64       
 9   postal_code                 category      
 10  year                        category      
 11  month                       category      
 12  day                         category      
 13  day_of_week_name_Start      object        
 14  Count_Charging              int64         
 15  Charging_time_in_hours_log  float64       
 16  Temperature       

In [9]:
data = df.copy()

In [10]:
#df.to_pickle("stat-v1.pickle")

In [11]:
#df.to_csv("stat-v1.csv")

# Pooled Ols

In [12]:
year = pd.Categorical(data.Year)
data = data.set_index(["evse_id","Year"])

In [13]:
data["Year"] = year

In [14]:
print(data.describe())

           latitude     longitude  Count_Charging  Charging_time_in_hours_log  \
count  1.796287e+06  1.796287e+06    1.796287e+06                1.796283e+06   
mean   4.988736e+01  9.604686e+00    3.196620e+03                1.421231e+00   
std    1.710223e+00  2.048584e+00    2.740239e+03                1.858277e+00   
min    4.685810e+01  5.901500e+00    1.000000e+00               -8.188689e+00   
25%    4.816250e+01  7.804500e+00    9.350000e+02                6.578078e-01   
50%    4.997890e+01  9.941000e+00    2.224000e+03                1.854691e+00   
75%    5.122210e+01  1.150950e+01    5.086000e+03                2.955279e+00   
max    5.490750e+01  1.466150e+01    1.079600e+04                3.178042e+00   

        Temperature  Precipitation            pd       Bahnhof    Restaurant  \
count  1.796287e+06   1.796287e+06  1.796287e+06  1.796287e+06  1.796287e+06   
mean   9.618019e+00   8.810534e-02  1.039320e+01  7.818120e+00  2.495758e-02   
std    3.029030e+00   4.620769

In [15]:
print(data.head())

                                                                               park_id  \
evse_id               Year                                                               
hj_DE*BLN*E*174060033 2019-12-17 09:00:13.369  hj_31c455b8-420e-46af-bce6-436b65ccc416   
                      2019-12-17 09:00:13.369  hj_31c455b8-420e-46af-bce6-436b65ccc416   
                      2019-12-17 09:00:13.369  hj_31c455b8-420e-46af-bce6-436b65ccc416   
                      2019-12-17 09:58:28.352  hj_31c455b8-420e-46af-bce6-436b65ccc416   
                      2019-12-17 09:58:28.352  hj_31c455b8-420e-46af-bce6-436b65ccc416   

                                               clc_code     area_code  \
evse_id               Year                                              
hj_DE*BLN*E*174060033 2019-12-17 09:00:13.369  Suburban  120650165165   
                      2019-12-17 09:00:13.369  Suburban  120650165165   
                      2019-12-17 09:00:13.369  Suburban  120650165165   
    

### PooledOLS is just plain OLS that understands that various panel data structures. It is useful as a base model

In [16]:
import statsmodels.api as sm
from linearmodels.panel import PooledOLS
from linearmodels.panel import PanelOLS

In [17]:
exog_vars = ["Temperature","pd","Precipitation","day_of_week_name_Start","month","year","clc_code","Restaurant", "Biergarten", "Community_Centre", "Hospital", "Kino", "Parking" ,"university","Bahnhof"]
exog = sm.add_constant(data[exog_vars])


/home/col-gku/.local/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [19]:
mod1 = PanelOLS(data.Charging_time_in_hours_log, exog)
panel_res1 = mod1.fit(cov_type='clustered', cluster_entity=True)
#print(panel_res1)
print((panel_res1.summary).as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}                       & Charging\_time\_in\_hours\_log & \textbf{  R-squared:         }   &      0.0202      \\
\textbf{Estimator:}                           &            PanelOLS            & \textbf{  R-squared (Between):}  &     -0.0642      \\
\textbf{No. Observations:}                    &            1796283             & \textbf{  R-squared (Within):}   &      0.0069      \\
\textbf{Date:}                                &        Wed, Jan 26 2022        & \textbf{  R-squared (Overall):}  &      0.0202      \\
\textbf{Time:}                                &            12:09:11            & \textbf{  Log-likelihood     }   &    -3.644e+06    \\
\textbf{Cov. Estimator:}                      &           Clustered            & \textbf{                     }   &                  \\
\textbf{}                                     &                                & \textbf{  F-statistic:       }   &      1122.2      \\
\t

In [30]:
mod1 = PooledOLS(data.Charging_time_in_hours_log, exog)
pooled_res1 = mod1.fit()
print(pooled_res1)

                              PooledOLS Estimation Summary                              
Dep. Variable:     Charging_time_in_hours_log   R-squared:                        0.0202
Estimator:                          PooledOLS   R-squared (Between):             -0.0642
No. Observations:                     1796283   R-squared (Within):               0.0069
Date:                        Mon, Jan 17 2022   R-squared (Overall):              0.0202
Time:                                15:31:28   Log-likelihood                -3.644e+06
Cov. Estimator:                    Unadjusted                                           
                                                F-statistic:                      1122.2
Entities:                                1877   P-value                           0.0000
Avg Obs:                               957.00   Distribution:              F(33,1796249)
Min Obs:                               1.0000                                           
Max Obs:             

In [17]:
import sys
filename  = open("pooled_results",'w')
sys.stdout = filename
print(pooled_res1)

# Random Effects

The random effects model is virtually identical to the pooled OLS model except that is accounts for the structure of the model and so is more efficient. Random effects uses a quasi-demeaning strategy which subtracts the time average of the within entity values to account for the common shock.

In [18]:
from linearmodels.panel import RandomEffects

mod2 = RandomEffects(data.Charging_time_in_hours_log, exog)
re_res = mod2.fit()
print(re_res)

In [17]:
re_res.variance_decomposition

Effects                   0.534856
Residual                  3.236605
Percent due to Effects    0.141817
Name: Variance Decomposition, dtype: float64

In [18]:
re_res.theta.head(10)

,theta
evse_id,
hj_DE*BLN*E*174060033,0.690202
hj_DE*BLN*E*174060031,0.682558
hj_DE*CON*E1000003,0.875920
hj_DE*ISE*E001126,0.928840
sl_23897984,0.838514
sl_23897986,0.697319
sl_23897988,0.809673
sl_23898096,0.863136
sl_23898098,0.690202


# The between estimator

The between estimator is an alternative, usually less efficient estimator, can can be used to estimate model parameters. It is particular simple since it first computes the time averages of and

and then runs a simple regression using these averages.

The year dummies are dropped since the averaging removes differences due to the year. expersq was also dropped since it is fairly co-linear with exper. These results are broadly similar to the previous models.

In [19]:
exog_vars1 = ["clc_code", "Restaurant", "Biergarten", "Community_Centre", "Hospital", "Kino", "Parking","university"]
exog = sm.add_constant(data[exog_vars1])

/home/col-gku/.local/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [20]:
from linearmodels.panel import BetweenOLS
mod3 = BetweenOLS(data.Charging_time_in_hours_log, exog)
be_res = mod3.fit(weights=True,check_rank=False)
print(be_res)

/home/col-gku/.local/lib/python3.8/site-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                             BetweenOLS Estimation Summary                              
Dep. Variable:     Charging_time_in_hours_log   R-squared:                        0.0451
Estimator:                         BetweenOLS   R-squared (Between):              0.0451
No. Observations:                        1877   R-squared (Within):               0.0000
Date:                        Sun, Jan 02 2022   R-squared (Overall):             -0.0413
Time:                                13:57:11   Log-likelihood                   -2188.5
Cov. Estimator:                    Unadjusted                                           
                                                F-statistic:                      8.8097
Entities:                                1877   P-value                           0.0000
Avg Obs:                               957.00   Distribution:                 F(10,1866)
Min Obs:                               1.0000                                           
Max Obs:             

# Panel Effects

In [21]:
from linearmodels.panel import PanelOLS

exog_vars2 = ["clc_code", "Restaurant", "Biergarten", "Community_Centre", "Hospital", "Kino", "Parking","university"]
exog = sm.add_constant(data[exog_vars2])




##### Time-invariant variables cannot be included when using entity effects since, once demeaned, these will all be 0

In [22]:
mod = PanelOLS(data.Charging_time_in_hours_log, exog, entity_effects=True,drop_absorbed=True)
fe_res = mod.fit()
print(fe_res)

/home/col-gku/.local/lib/python3.8/site-packages/linearmodels/panel/model.py:1796: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

clc_code.Uninhabited, clc_code.Urban, Community_Centre, Hospital, Kino, university

  warnings.warn(


                              PanelOLS Estimation Summary                               
Dep. Variable:     Charging_time_in_hours_log   R-squared:                      1.11e-16
Estimator:                           PanelOLS   R-squared (Between):             -2.6171
No. Observations:                     1796283   R-squared (Within):               0.0000
Date:                        Sun, Jan 02 2022   R-squared (Overall):             -0.2884
Time:                                13:57:25   Log-likelihood                -3.607e+06
Cov. Estimator:                    Unadjusted                                           
                                                F-statistic:                   7.162e-11
Entities:                                1877   P-value                           1.0000
Avg Obs:                               957.00   Distribution:               F(4,1794402)
Min Obs:                               1.0000                                           
Max Obs:             

/home/col-gku/.local/lib/python3.8/site-packages/linearmodels/panel/results.py:85: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


Time Effects¶

Time effect can be added using time_effects=True. Here the time dummies are removed. Note that the core coefficients are identical. The only change is in the test statistic for poolability since not the “effects” include both entity and time, whereas before only entity were included.

In [44]:
exog_vars_1 = ["Restaurant", "Biergarten", "Parking"]
exog1 = sm.add_constant(data[exog_vars_1])
mod = PanelOLS(data.Charging_time_in_hours_log, exog1, entity_effects=True, time_effects=True)
fe_te_res = mod.fit()
print(fe_te_res)

NameError: name 'PanelOLS' is not defined